In [72]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/19 10:05
# @Author  : Wang Yujia
# @File    : data_extract_for_targets.ipynb
# @Description : 为了target datas提取信息，而且提取data_key

# 1. Preparations
## 1.1 全局设置
1. input的data包括那些duration过长的结果
2. output一共`data_key.shape[0]`+1个文件

In [73]:
data_path = "../data/datawithnp_2_selected.csv"
prod_embedding_output_path = "../data/prod_embedding.csv"

# output path
target_output_head= "../data/targets/target_data_NP_"
target_output_tail= ".csv"
target_data_key_path = "../data/target_datakey.csv"

features = ['id', 'bidincrement', 'bidfee','retail']
import pandas as pd
import numpy as np
from visdom import Visdom
import csv
from tqdm import tqdm
from IPython.display import clear_output

## 1.2 data读取与保存data_key

In [74]:
data = pd.read_csv(data_path, encoding="utf-8")
data.head(),data.shape

(   auction_id  product_id                                      item  \
 0       86827    10009602  sony-ericsson-s500i-unlocked-mysterious-   
 1       87964    10009881            psp-slim-lite-sony-piano-black   
 2       87965    10009881            psp-slim-lite-sony-piano-black   
 3       88638    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 4       88639    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 
                                             desc  retail  price  finalprice  \
 0  Sony Ericsson S500i Unlocked Mysterious Green  499.99  13.35       13.35   
 1               PSP Slim & Lite Sony Piano Black  169.99  74.70       74.70   
 2               PSP Slim & Lite Sony Piano Black  169.99  83.10       83.10   
 3  Sony Ericsson S500i Unlocked Mysterious Green  499.99  19.65       19.65   
 4  Sony Ericsson S500i Unlocked Mysterious Green  499.99  47.10       47.10   
 
    bidincrement  bidfee        winner  ...           endtime_str  \
 0          0.1

In [75]:
data_key = data[features].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
# make sure int
data_key['id'] = data_key['id'].astype(int)
data_key.to_csv(target_data_key_path,header=True, encoding="utf-8",index=False)
print(f"target data中，一共包括 *{data_key.shape[0]}* 个data keys")
data_key.head(),data_key.shape

target data中，一共包括 *1196* 个data keys


(   id  bidincrement  bidfee  retail
 0   0          0.15    0.75  499.99
 1   1          0.15    0.75  169.99
 2   2          0.15    0.75  299.99
 3   3          0.15    0.75   89.99
 4   5          0.15    0.75   59.99,
 (1196, 4))

## 1.3 functions about 'key'

In [76]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))   # 'product_id'是int类型
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

def select_data_fromkey(key_i_str):
    return data[(data['id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. 整理与输出

1. 整理成一个[key]对应一个“N” vector的格式
    - 最后的data中不包括"cnt_n_2"这一列，
2. 注意一定要按照`data_key`逐行保存csv，这样index可以对应


In [77]:
plot_flag = 0
if(plot_flag):
    env_str = 'N_targets'
    viz = Visdom(env=env_str)

# col_names = ['id','bidincrement','bidfee','retail','N','P','cnt_n_2']
col_names = ['N','P']

data_key.shape[0]

1196

In [79]:
# data_key.shape[0]
for i in tqdm(range(0,data_key.shape[0])):

    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    data_i_df = data_i[col_names]

    # data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt_n_2'])]
    # get the filepath and output
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# clear_output()
print("Done")

100%|██████████| 1196/1196 [00:03<00:00, 347.15it/s]

Done


# 4 save
1. 保存结果以及viz环境

In [ ]:
# 保存整个环境
if(plot_flag):
    viz.save(envs=[env_str])